<a href="https://www.kaggle.com/code/aqilwahid/smart-agents-using-bfs?scriptVersionId=197857528" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# 1. Install Depedensi

In [1]:
!pip install ipywidgets pandas -q
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


# 2. Import Modul

In [2]:
import numpy as np
import os 
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# 3. Membaca Data

In [3]:
# Memuat dataset
data = pd.read_csv('../input/tictactoe/Tic tac initial results.csv', delimiter=',')
data.dataframeName = 'Tic tac initial results.csv'

### 3.1. Visualisasi Data

In [4]:
nRow, nCol = data.shape
print(f'Terdapat {nRow} baris and {nCol} kolom')

data.head(5)

Terdapat 2436 baris and 8 kolom


,MOVE1,MOVE2,MOVE3,MOVE4,MOVE5,MOVE6,MOVE7,CLASS
0,0,8,1,3,?,?,?,loss
1,4,7,2,6,?,?,?,win
2,0,8,1,6,5,?,?,draw
3,4,7,2,3,?,?,?,draw
4,0,4,2,1,?,?,?,win


### 3.2 Preproses Data

In [5]:
# Mengganti '?' dengan nilai NaN
data = data.replace('?', np.nan)

# Mengubah kolom MOVE menjadi numerik, nilai yang tidak bisa dikonversi akan menjadi NaN
for col in ['MOVE1', 'MOVE2', 'MOVE3', 'MOVE4', 'MOVE5', 'MOVE6', 'MOVE7']:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Memfilter data kemenangan
winning_data = data[data['CLASS'] == 'win']

# Membuat daftar jalur kemenangan
winning_paths = []

for index, row in winning_data.iterrows():
    moves = row[['MOVE1', 'MOVE2', 'MOVE3', 'MOVE4', 'MOVE5', 'MOVE6', 'MOVE7']].dropna().astype(int).tolist()
    winning_paths.append(moves)

# 4. Implementasi BFS

In [6]:
def is_winner(board, player):
    win_conditions = [
        [board[0], board[1], board[2]],
        [board[3], board[4], board[5]],
        [board[6], board[7], board[8]],
        [board[0], board[3], board[6]],
        [board[1], board[4], board[7]],
        [board[2], board[5], board[8]],
        [board[0], board[4], board[8]],
        [board[2], board[4], board[6]],
    ]
    return [player, player, player] in win_conditions

def is_draw(board):
    return ' ' not in board and not is_winner(board, 'X') and not is_winner(board, 'O')

def get_available_moves(board):
    return [i for i, spot in enumerate(board) if spot == ' ']

def apply_move(board, move, player):
    new_board = board.copy()
    new_board[move] = player
    return new_board

In [7]:
import collections

def bfs_agent_with_dataset(board, player, winning_paths):
    opponent = 'O' if player == 'X' else 'X'
    queue = collections.deque()
    queue.append((board.copy(), []))
    visited = set()

    def board_to_tuple(b):
        return tuple(b)

    while queue:
        current_board, path = queue.popleft()
        board_key = board_to_tuple(current_board)

        if board_key in visited:
            continue
        visited.add(board_key)

        if is_winner(current_board, player):
            if path:
                return path[0]
            else:
                continue

        if is_draw(current_board) or is_winner(current_board, opponent):
            continue

        possible_moves = get_available_moves(current_board)

        prioritized_moves = []

        for move in possible_moves:
            move_in_winning_paths = False
            for winning_path in winning_paths:
                if len(path) < len(winning_path) and move == winning_path[len(path)]:
                    move_in_winning_paths = True
                    break
            if move_in_winning_paths:
                prioritized_moves.append((0, move))
            else:
                prioritized_moves.append((1, move))

        prioritized_moves.sort()

        for _, move in prioritized_moves:
            new_board = current_board.copy()
            new_board[move] = player
            queue.append((new_board, path + [move]))

    moves = get_available_moves(board)
    return moves[0] if moves else None

# 5. Implementasi Antarmuka IPyWidgets

In [8]:
# State permainan
current_board = [' ' for _ in range(9)]
current_player = 'X'  # Pemain manusia
agent_player = 'O'    # Agen
game_over = False

# Output widgets
board_output = widgets.Output()
message_output = widgets.Output()

In [9]:
def render_board():
    with board_output:
        clear_output()
        board = current_board
        button_layout = widgets.Layout(width='60px', height='60px')
        buttons = []
        for i in range(9):
            if board[i] == ' ' and not game_over:
                btn = widgets.Button(description=' ', layout=button_layout)
                btn.on_click(lambda x, idx=i: player_move(idx))
            else:
                btn = widgets.Button(description=board[i], layout=button_layout, disabled=True)
            buttons.append(btn)
        grid = widgets.GridBox(buttons, layout=widgets.Layout(
            grid_template_columns='repeat(3, 60px)',
            grid_template_rows='repeat(3, 60px)',
            grid_gap='5px 5px'
        ))
        display(grid)

In [10]:
def player_move(cell_index):
    global current_board, current_player, agent_player, game_over

    if game_over:
        return

    if current_board[cell_index] != ' ':
        with message_output:
            clear_output()
            print("Sel sudah terisi. Pilih sel lain.")
        return

    current_board[cell_index] = current_player

    if is_winner(current_board, current_player):
        game_over = True
        with message_output:
            clear_output()
            print("Selamat! Anda menang!")
        render_board()
        return
    elif is_draw(current_board):
        game_over = True
        with message_output:
            clear_output()
            print("Permainan berakhir seri!")
        render_board()
        return

    # Agen melakukan gerakan
    agent_move_index = bfs_agent_with_dataset(current_board, agent_player, winning_paths)
    if agent_move_index is not None:
        current_board[agent_move_index] = agent_player

        if is_winner(current_board, agent_player):
            game_over = True
            with message_output:
                clear_output()
                print("Maaf, Anda kalah. Agen menang.")
            render_board()
            return
        elif is_draw(current_board):
            game_over = True
            with message_output:
                clear_output()
                print("Permainan berakhir seri!")
            render_board()
            return
    else:
        # Jika agen tidak dapat bergerak
        if is_draw(current_board):
            game_over = True
            with message_output:
                clear_output()
                print("Permainan berakhir seri!")
            render_board()
            return

    with message_output:
        clear_output()
        print("Giliran Anda.")
    render_board()

In [11]:
def reset_game(b):
    global current_board, current_player, agent_player, game_over
    current_board = [' ' for _ in range(9)]
    current_player = 'X'
    agent_player = 'O'
    game_over = False
    with message_output:
        clear_output()
        print("Permainan dimulai. Giliran Anda.")
    render_board()

# 6. Hasil

In [12]:
with message_output:
    print("Permainan dimulai. Giliran Anda.")

reset_button = widgets.Button(description='Reset Permainan', button_style='info')
reset_button.on_click(reset_game)

display(message_output)
display(board_output)
render_board()
display(reset_button)

Output()

Output()

Button(button_style='info', description='Reset Permainan', style=ButtonStyle())